在 MMDeploy 本地预测步骤，因 miniconda 中的 PIL 问题，导致后续步骤无法进行。
```
importerror: cannot import name 'is_path' from 'pil._util' (/root/miniconda3/lib/python3.8/site-packages/pil/_util.py)
```

以下为模型训练的评估指标，及模型转换后的预测结果：

### 模型评估指标
- mmdetection
![](./static/test_mmdet.jpg)

- mmpose
![](./static/test_mmpose.jpg)

- 原始图片（图片已加微信ID水印）

![](./static/ear.jpg)

- rtmdet2onnx 预测图片

![](./rtmdet2onnx/output_onnxruntime.jpg)

- rtmpose2onnx 预测图片

![](./rtmpose2onnx/output_onnxruntime.jpg)

模型训练结果见本仓库中的`rtmdet2onnx`，`rtmpose2onnx`两个文件夹

---

# 过程笔记：

## 环境配置
完成 MMPose 和 MMDetection 及相关依赖的安装，验证如下（如无报错，则说明环境安装成功）：

In [ ]:
# 检查 Pytorch
import torch, torchvision
print('Pytorch 版本', torch.__version__)
print('CUDA 是否可用',torch.cuda.is_available())

# 检查 mmcv
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('MMCV版本', mmcv.__version__)
print('CUDA版本', get_compiling_cuda_version())
print('编译器版本', get_compiler_version())

# 检查 mmpose
import mmpose
print('mmpose版本', mmpose.__version__)

# 检查 mmdet
import mmdet
print('mmdetection版本', mmdet.__version__)

## 准备数据
在命令行中使用 unzip 命令解压 Ear210_Dataset_coco.zip 文件，然后用 rm 删除。

## 准备 Config 文件
将准备好的 rtmdet_tiny_ear.py 和 rtmpose-s-ear.py 放到 data 路径下。

## MMDet 训练
将上一步中的 data 文件替换到 mmdetection 路径下。
结构如下：
```
mmdetection/data/
    Ear210_Keypoint_Dataset_coco/
    rtmdet_tiny_ear.py
    rtmpose-s-ear.py
```

In [4]:
# 切换路径，并确认路径在 mmdetection
import os

os.chdir('mmdetection')
os.getcwd()

'/root/autodl-tmp/MMPose_Tutorials/2023/0524/mmdetection'

在命令行中执行：
```
python tools/train.py data/rtmdet_tiny_ear.py
```

## MMPose 训练
同理，切换到 mmpose 路径下后，执行训练

In [ ]:
os.chdir('mmpose')
os.getcwd

如果提示缺少 albumentations 库，执行以下命令安装：
```
pip install -U albumentations
```

训练过程中，可能会因 np.float 报错，执行以下命令重新安装：
```
pip install "numpy<1.24.0"
```

然后在命令行中执行：
```
python tools/train.py data/rtmpose-s-ear.py
```

## 评估测试集
分别在测试集上评估模型精度：

mmdetection
```
python tools/test.py data/rtmdet_tiny_ear.py \
                      work_dirs/rtmdet_tiny_ear/best_coco_bbox_mAP_epoch_199.pth
```

mmpose
```
python tools/test.py data/rtmpose-s-ear.py \
                      work_dirs/rtmpose-s-ear/best_PCK_epoch_300.pth
```

## 模型转换
分别转换 mmdet 和 mmpose：

mmdetection
```
python tools/model_converters/publish_model.py \
        work_dirs/rtmdet_tiny_ear/best_coco_bbox_mAP_epoch_199.pth \
        checkpoint/fixed_rtmdet.pth
```

mmpose
```
python tools/misc/publish_model.py \
        work_dirs/rtmpose-s-ear/best_PCK_epoch_300.pth \
        checkpoint/fixed_rtmpose.pth
```

## 模型部署
### 环境配置
```
pip install mmdeploy==1.0.0 onnxruntime-gpu
git clone https://github.com/open-mmlab/mmdeploy.git
```

In [8]:
## 验证安装
import mmdeploy
print('MMDeploy 版本', mmdeploy.__version__)

MMDeploy 版本 1.0.0


### 模型转ONNX

In [10]:
import os
os.chdir('../mmdeploy')

mmdetection:

In [ ]:
!python tools/deploy.py \
        configs/mmdet/detection/detection_onnxruntime_dynamic.py \
        ../mmdetection/data/rtmdet_tiny_ear.py \
        ../mmdetection/checkpoint/fixed_rtmdet-3378ee08.pth \
        ../mmdetection/data/test/ear.jpg \
        --work-dir ../rtmdet2onnx \
        --dump-info

mmpose:

In [ ]:
!python tools/deploy.py \
        configs/mmpose/pose-detection_simcc_onnxruntime_dynamic.py \
        ../mmpose/data/rtmpose-s-ear.py \
        ../mmpose/checkpoint/fixed_rtmpose-106c19a1_20230604.pth \
        ../mmpose/data/test/ear.jpg \
        --work-dir ../rtmpose2onnx \
        --dump-info

## 模型预测-单张图片

### 环境配置
```
pip install mmdeploy-runtime-gpu==1.0.0
pip install matplotlib tqdm opencv-python tqdm
pip install onnxruntime-gpu==1.8.1
```
### 载入模型

In [27]:
os.chdir('../')
os.getcwd()

'/root/autodl-tmp/MMPose_Tutorials/2023/0524'

In [29]:
import cv2
import numpy as np
import time
from tqdm import tqdm

from mmdeploy_runtime import Detector
from mmdeploy_runtime import PoseDetector

# import matplotlib.pyplot as plt

# 载入单张图片
img_path = 'single_image/ear.jpg'
img_bgr = cv2.imread(img_path)

# 目标检测模型目录
detect = 'rtmdet2onnx'

# 关键点检测模型目录
pose = 'rtmpose2onnx'

# 计算设备
device = 'cuda'

bbox_detector = Detector(detect, device)
pose_detector = PoseDetector(pose, device)

[2023-06-04 08:17:31.106] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "rtmdet2onnx"
[2023-06-04 08:17:31.107] [mmdeploy] [error] [net_module.cpp:47] Net backend not found: onnxruntime, available backends: []
[2023-06-04 08:17:31.107] [mmdeploy] [error] [task.cpp:99] error parsing config: {
  "context": {
    "device": "<any>",
    "model": "<any>",
    "stream": "<any>"
  },
  "input": [
    "prep_output"
  ],
  "input_map": {
    "img": "input"
  },
  "is_batched": true,
  "module": "Net",
  "name": "rtmdet",
  "output": [
    "infer_output"
  ],
  "output_map": {},
  "type": "Task"
}
[2023-06-04 08:17:31.107] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "rtmpose2onnx"
[2023-06-04 08:17:31.107] [mmdeploy] [error] [net_module.cpp:47] Net backend not found: onnxruntime, available backends: []
[2023-06-04 08:17:31.107] [mmdeploy] [error] [task.cpp:99] error parsing config: {
  "context": {
    "device": "<any>",
    "model": "<any>",
    "stream": "<any